# **Classifying coronary artery disease using age, and blood cholesterol, and maximum heart rate.**
## **Group Report**

#### Zirun Xu, Aura Balita, and Sahil Babani

## Introduction
Coronary artery disease (CAD) is a cardiovascular condition caused by the accumulation of plaque in the coronary arteries, which are responsible for delivering blood to the heart. This plaque buildup gradually narrows the arteries, leading to reduced blood flow to the heart. CAD can lead to abnormal heart rhythms, heart attacks, and heart failure (Cleveland Clinic, n.d.). Four factors that increase the likelihood of heart disease are maximum heart rate, blood pressure, and blood cholesterol levels. Individuals aged 65 and older are more likely to develop CAD due to the buildup of plaque over the years (National Institute on Aging, n.d.), while elevated blood pressure and high blood cholesterol levels are indicative of narrowed blood vessels due to plaque accumulation (Centers for Disease Control and Prevention, n.d.). Lastly, the max heart rate achieved during exercise testing is typically indicative of how well the heart is functioning. A high maximum heart rate usually means a healthier heart, while a lower one can signify a less healthy heart.).
This project aims to determine whether it is possible to predict the likelihood of an individual having heart disease based on their age, blood pressure, blood cholesterol, and maximum heart rate and to determine which variable pairs would best give the best predictions, using the Cleveland database.
This database was constructed by gathering clinical test results from patients with chest pain symptoms at the Cleveland Clinic in Cleveland, Ohio database (Detrano, et al., 89).

## Methods & Results

(Explain dataset that will be used)\
(Copy and paste variables)\
(Describe the methods and approaches used to collect data)
* Explain data collection techniques
* Explain analytical techinques

### Loading the file and exploring the data
The first step is to load the `tidyverse` and `tidymodels` packages that will be used for the analysis of our data.

In [ ]:
# Importing packages
library(tidyverse)
library(tidymodels)

Our heart disease dataset is a `.csv` file with no headers. The `read_csv` function will be used to load the data, and headers will be added using the `colnames()` function. The data will then be inspected.

In [ ]:
# Loading and adding headers to the heart disease data
heart <- read_csv("processed.cleveland.data")
colnames(heart) <- c("age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", 
                     "thalach", "exang", "oldpeak", "slope", "ca", "thal", "num")
head(heart)

### Describing the variables in the heart disease data set

Patients experiencing chest pain symptoms, also known as angina, often have a higher probability of being diagnosed with heart disease and are often recommended for coronary angiography. An angiogram involves using x-ray imaging to examine the blood vessels in a person's heart. Its purpose is to assess the condition of these vessels and the circulation of blood within them. At the Cleveland Clinic, 303 patients undergoing coronary angiography had 14 different variables measured.

The information gathered included four clinical details: age, sex, type of chest pain, and systolic blood pressure. Non-invasive tests involved drawing blood to measure cholesterol levels and blood sugar, electrocardiograms, in which the electrical activity of the heart is measured, thallium scans, in which a radioactive tracer is used to see how much blood is reaching different parts of your heart, and fluoroscopy for coronary calcium, which is a procedure used to look at calcium in the heart's blood vessels.

These variables are described below:

1. **age:** age in years
2. **sex:** sex of the individual (1 = male; 0 = female)
3. **cp:** chest pain type
4. **trestbps:** resting blood pressure (in mm Hg on admission to the hospital)
5. **chol:** serum cholesterol in mg/dl
6. **fbs:** whether fasting blood sugar is greater than 120 mg/dl (1 = true; 0 = false)
7. **restecg:** resting electrocardiographic results
8. **thalach:** maximum heart rate achieved (in bpm)
9. **exang:** exercise induced angina (1 = yes; 0 = no)
10. **oldpeak:** ST depression induced by exercise relative to rest
11. **slope:** the slope of the peak exercise ST segment
12. **ca:** number of major vessels (0-3) colored by flourosopy
13. **thal:** thalassemia
14. **num:** diagnosis of heart disease (0 = not present; 1, 2, 3, 4 = present)

As mentioned in the introduction, we will only be using age, blood pressure, blood cholesterol, and maximum heart rate for our analysis. Thus, we will create a new tibble using these four variables using `select()`.

### Wrangling and cleaning the data

In [ ]:
# wrangles and cleans the data from it's original (downloaded) format to the format necessary for the planned analysis

### Summary of the data

In [ ]:
# performs a summary of the data set that is relevant for exploratory data analysis related to the planned analysis

### Splitting the data

(explain why it's important to split the data)

In [ ]:
# add code here

### Preliminary data visualization

In [ ]:
# create a visualization of the dataset that is relezant for exploratory data analysis related to the planned analysis

### Determining the most suitable $k$ value

In [ ]:
# add code here

### Plotting $k$ vs Accuracy

In [ ]:
# add code here

(explain which k should be used based on plot)

### Creating a $k$-NN classification model

In [ ]:
# add code here

### Testing the model against another dataset

In [ ]:
# add code here

## Discussion
- summarize what you found
- discuss whether this is what you expected to find
- discuss what impact could such findings have
- discuss what future questions could this lead to

## Conclusions

## References